Importing Libraries and dataset

In [1]:
import pandas as pd 
import numpy as np 
import sqlite3 as sq
from bs4 import BeautifulSoup

In [2]:
conn = sq.connect("../database.sqlite")
data = pd.read_sql_query('SELECT * FROM REVIEWS WHERE SCORE!=3 ',conn)  #assuming 3 as neutral reviews and hence removing the same from our datastet
print(data.head())
print(data.shape)

   Id   ProductId          UserId                      ProfileName   
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian  \
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time   
0                     1                       1      5  1303862400  \
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [3]:
#count of rows having neutral rows
print("original_data_shape =",pd.read_sql_query('SELECT COUNT(*) FROM REVIEWS',conn))
print("neutral_reviews_count = ",pd.read_sql_query('Select count(*) from REVIEWS WHERE SCORE=3',conn))

#checking for redundancy where HelpFulnessNumerator is greater than HelpfulnessDenominator
print(pd.read_sql_query('select  * from REVIEWS WHERE HelpFulnessNumerator>HelpFulnessDenominator',conn).shape)  
conn.close()         #closing DB connection

original_data_shape =    COUNT(*)
0    568454
neutral_reviews_count =     count(*)
0     42640
(2, 10)


In [4]:
# score of > 3 is positive and <3 taken as negative
def re_score(x):
    if x < 3: 
        return 0
    return 1

data['Score'] = data['Score'].map(re_score)
data.head() 

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
#kind = type of sorting (read np.sort)
#position of null positions (na_postion) = first or last
sorted_data = data.sort_values('ProductId',axis='index',ascending=True,kind='mergesort',na_position='last')

In [6]:
df = sorted_data.drop_duplicates(subset={"ProductId","UserId","Time","ProfileName","Text"},keep='first')
df.shape

(524691, 10)

In [7]:
df = df[df['HelpfulnessNumerator']<=df['HelpfulnessDenominator']]  #two records were present
df.shape


(524689, 10)

In [8]:
df.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138676,150493,0006641040,AMX0PJKV4PPNJ,"E. R. Bird ""Ramseelbird""",71,72,1,1096416000,Read it once. Read it twice. Reading Chicken S...,"These days, when a person says, ""chicken soup""..."
138677,150494,0006641040,AYZ0PR5QZROD1,Mother of 3 girls,3,3,1,1173312000,Family favorite,All of my children love this book. My first g...
138678,150496,0006641040,A3KKR87BJ0C595,"Gretchen Goodfellow ""Lover of children's lit""",3,3,1,1111363200,"You'll use it once, you'll use it twice",One of my earliest memories is of this book. ...
138679,150497,0006641040,A1HKYQOFC8ZZCH,"Maria Apolloni ""lanarossa""",2,2,0,1334707200,"The story is great, the softcover book is disa...",I give five stars to the Maurice Sendak story....
138680,150498,0006641040,A3SJWISOCP31TR,R. J. Wells,2,2,1,1176336000,A Gem of a Book,This is a wonderful little book. I loved it 40...


In [9]:
# checking for redundant values
print(df.isna().any())

Id                        False
ProductId                 False
UserId                    False
ProfileName               False
HelpfulnessNumerator      False
HelpfulnessDenominator    False
Score                     False
Time                      False
Summary                   False
Text                      False
dtype: bool


Text Preprocessing

In [10]:
import re

In [11]:
text1 = df['Text'].values[0]
text1

'These days, when a person says, "chicken soup" they\'re probably going to follow up those words with, "for the soul" or maybe "for the teenaged soul".  Didn\'t used to be that way.  Why I can remember a time when if a person said, "chicken soup" those words were followed by an enthusiastic "with rice!".  Such was the power of Maurice Sendak\'s catchy 1962 children\'s book.  I am pleased to report that if you care to read this book again today, you will find it hasn\'t dimished a jot in terms of frolicksome fun.  In this book we are led through a whirlwind chicken soup year with our host, a boy who bears no little resemblance to Sendak\'s other great rhyming tale "Pierre" (in looks if not demeanor).  It\'s a catchy flouncy bouncy combo of soup and the people who love it so.<br /><br />This is ostensibly a book meant to teach your children the different months of the year.  Each month gets its own rhythmic poem and accompanying illustration.  These are fairly simple pen and ink drawings

In [12]:
#removing URLs from text
text0  = "www.pornhub is my favourite web"
text0 = re.sub(r'http\S+','',text0)
text0 = re.sub(r'www\S+','',text0)
text0

' is my favourite web'

In [13]:
#removing HTML tags
from bs4 import BeautifulSoup
text2 = "<html><body><p>We have used the Victor fly bait for 3 seasons.  Can't beat it.  Great product!</p></body></html>"
text2 = BeautifulSoup(text2).get_text()
text2


"We have used the Victor fly bait for 3 seasons.  Can't beat it.  Great product!"

In [14]:

#Removing special character and punctuations
text1 = '''These days, when a person says, "chicken soup" they\'re 
probably going to follow up those words with, "for the soul" or maybe "for the teenaged soul".  
Didn\'t used to be that way.  Why I can remember a time when if a person said, "chicken soup" 
those words were followed by an enthusiastic "with rice!".  Such was the power of Maurice Sendak\'s catchy 
1962 children\'s book.  I am pleased to report that if you care to read this book again today, 
you will find it hasn\'t dimished a jot in terms of frolicksome fun.  
In this book we are led through a whirlwind chicken soup year with our host, 
a boy who bears no little resemblance to Sendak\'s other great rhyming tale "Pierre" (in looks if not demeanor).  
It\'s a catchy flouncy bouncy combo of soup and the people who love it so.<br /><br />This is ostensibly a book 
meant to teach your children the different months of the year.  Each month gets its own rhythmic poem and accompanying illustration.  
These are fairly simple pen and ink drawings with the occasional splash of blue (in varying shades), yellow, gray, and green.  
You may wonder how an author could ever hope to come up with twelve highly original soup-related poems.  I mean, honestly, how much 
is there to say about even the fanciest soup, let alone chicken soup with rice?  Quite a lot, as it happens.  In the cold winter
 months soup is supped while sliding on ice, while celebrating the birthday of a snowman, and in a gusty gale as a whale. 
   In the spring there\'s robin\'s nest soup, soup to cure drooping roses, and soup stolen by jealous March winds.  
   Our hero postulates the potential joys that could come of being a cooking pot, stewing soup or (oddly enough) as 
   "a baubled bangled Christmas tree".<br /><br />Not to degrade the reading skills of parents everywhere,
     but I cannot recommend enough getting an audio version of this tale to accompany your child\'s reading. 
       Though I am now a wise and cultured 26 year-old (the years have been kind to me in this, my old age)
         I can still remember the chicken soup with rice tune.  Heck, I read this entire book recently and 
         found I could do the song perfectly with each and every line.  Now maybe you have your own particular chicken soup with rice song style that
           you\'re just loathe to give up.  If so, fine.  I understand why you might not want to taint your already existing chicken soup melody.
               But if you haven\'t found a jingle to accompany this book, get the audio version immediately,
                 if not sooner.  Until you can sing "Whoopy once, whoopy twice, whoopy chicken soup with rice" with the correct oomph,
                   you\'re missing out.<br /><br />I take my "Chicken Soup With Rice" readings seriously. 
                     This book was the "Chicka Chicka Boom Boom" of its day, and still remains the catchiest method to teach kids the months of the year.  
                     It is also seriously in danger of being forgotten.  So pull out your old accordion and strap on your dancing shoes.  
                     The time for yukkin\' it up to a merry dance of poultry broth is here.  It\'s Sendak at his finest.'''

text1 = re.sub('[^A-Za-z0-9]+',' ',text1)
text1

'These days when a person says chicken soup they re probably going to follow up those words with for the soul or maybe for the teenaged soul Didn t used to be that way Why I can remember a time when if a person said chicken soup those words were followed by an enthusiastic with rice Such was the power of Maurice Sendak s catchy 1962 children s book I am pleased to report that if you care to read this book again today you will find it hasn t dimished a jot in terms of frolicksome fun In this book we are led through a whirlwind chicken soup year with our host a boy who bears no little resemblance to Sendak s other great rhyming tale Pierre in looks if not demeanor It s a catchy flouncy bouncy combo of soup and the people who love it so br br This is ostensibly a book meant to teach your children the different months of the year Each month gets its own rhythmic poem and accompanying illustration These are fairly simple pen and ink drawings with the occasional splash of blue in varying sha

In [15]:
#removing alpha-numeric characters
text2 = re.sub('\S*\d\S*','',text2)
text2

"We have used the Victor fly bait for  seasons.  Can't beat it.  Great product!"

In [16]:
#performing decontraction
def decontraction(phrase):
    phrase = re.sub(r"won't","will not",phrase)
    phrase = re.sub(r"can't","can not",phrase)
    phrase = re.sub(r"n't","not",phrase)
    phrase = re.sub(r"\'re","are",phrase)
    phrase = re.sub(r"\'s"," is",phrase)
    phrase = re.sub(r"\'d"," would",phrase)
    phrase = re.sub(r"\'ll"," will",phrase)
    phrase = re.sub(r"\'t"," not",phrase)
    phrase = re.sub(r"\'ve"," have",phrase)
    phrase = re.sub(r"\'m"," am",phrase)
    return phrase


In [17]:
text4 = 'Mrs. May\'s pumpkin crunch is a great snack. For those of you watching calories and intake, be aware that the 2oz portions are two servings. Most of Mrs. May\'s products are great, but with the fat at 11g per serving, i find it\'s not a complete guilt free snack. However, the pumpkin seed crunch has 9gms of protein per serving vs. the usual 4gms in their other snacks. So if you\'re monitoring your intake and want to indulge in these now and then, the pumpkin seed crunch is the healthiest option of the bunch.<br /><br />That said, they are all tasty and certainly not bad for you, just a little high in fat if you\'re watching your weight.'
print(decontraction(text4))

Mrs. May is pumpkin crunch is a great snack. For those of you watching calories and intake, be aware that the 2oz portions are two servings. Most of Mrs. May is products are great, but with the fat at 11g per serving, i find it is not a complete guilt free snack. However, the pumpkin seed crunch has 9gms of protein per serving vs. the usual 4gms in their other snacks. So if youare monitoring your intake and want to indulge in these now and then, the pumpkin seed crunch is the healthiest option of the bunch.<br /><br />That said, they are all tasty and certainly not bad for you, just a little high in fat if youare watching your weight.


In [18]:
# converting texts into lower case
def lower_case(phrase):
    phrase = ''.join(i.lower() for i in phrase)
    return phrase

print(lower_case(text4))

mrs. may's pumpkin crunch is a great snack. for those of you watching calories and intake, be aware that the 2oz portions are two servings. most of mrs. may's products are great, but with the fat at 11g per serving, i find it's not a complete guilt free snack. however, the pumpkin seed crunch has 9gms of protein per serving vs. the usual 4gms in their other snacks. so if you're monitoring your intake and want to indulge in these now and then, the pumpkin seed crunch is the healthiest option of the bunch.<br /><br />that said, they are all tasty and certainly not bad for you, just a little high in fat if you're watching your weight.


In [19]:
#removing stopwords

import nltk 
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
def remove_stop_words(phrase):
    phrase  = " ".join(i for i in phrase.split() if i.lower() not in stopwords)
    return phrase

print(remove_stop_words(text4))

Mrs. May's pumpkin crunch great snack. watching calories intake, aware 2oz portions two servings. Mrs. May's products great, fat 11g per serving, find complete guilt free snack. However, pumpkin seed crunch 9gms protein per serving vs. usual 4gms snacks. monitoring intake want indulge then, pumpkin seed crunch healthiest option bunch.<br /><br />That said, tasty certainly bad you, little high fat watching weight.


In [20]:
#stemming words
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')


In [22]:
#single block to perform all text-preprocessing
from tqdm import tqdm

corpus = []
for review in tqdm(df['Text'].values) :
    review = re.sub(r'http\S+','',review)
    review = re.sub(r'www\S+','',review)
    review = re.sub('[^A-Za-z0-9]+'," ",review)
    review = BeautifulSoup(review).get_text()
    review = re.sub('\S*\d\S*',' ',review)
    review = ' '.join(i.lower() for i in review.split() if i.lower() not in stopwords)
    review = decontraction(review)
    review = ' '.join(stemmer.stem(i) for i in review.split())
    corpus.append(review)



100%|██████████| 524689/524689 [06:48<00:00, 1283.77it/s]


In [23]:
corpus[999]

'whole idea keep flea hous invad tri everi thing spray power name temporari solut best best respons invas trap invad trap flea remov trap kid yes trap work remov flea dead chemic flea get hous prefer kill resid hous tri trap br br mike'

Vectorization

1. Bag of Words

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

def bag_of_words(data):
    bow = CountVectorizer()
    return bow.fit_transform(data)
    

2. TF-IDF

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idf(data):
    cv= TfidfVectorizer()
    return cv.fit_transform(data).toarray()


3.  Word2vec

In [45]:
data = corpus[0:10]
data

['day person say chicken soup probabl go follow word soul mayb teenag soul use way rememb time person said chicken soup word follow enthusiast rice power mauric sendak catchi children book pleas report care read book today find dimish jot term frolicksom fun book led whirlwind chicken soup year host boy bear littl resembl sendak great rhyme tale pierr look demeanor catchi flounci bounci combo soup peopl love br br ostens book meant teach children differ month year month get rhythmic poem accompani illustr fair simpl pen ink draw occasion splash blue vari shade yellow gray green may wonder author could ever hope come twelv high origin soup relat poem mean honest much say even fanciest soup let alon chicken soup rice quit lot happen cold winter month soup sup slide ice celebr birthday snowman gusti gale whale spring robin nest soup soup cure droop rose soup stolen jealous march wind hero postul potenti joy could come cook pot stew soup odd enough baubl bangl christma tree br br degrad re

In [132]:
X = bag_of_words(data)
X_bow = X.toarray()
print(X_bow.shape)
print(X_bow)

(10, 350)
[[0 0 3 ... 1 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [133]:
X_tfidf = tf_idf(data)
print(X_tfidf.shape)
print(X_tfidf)

(10, 350)
[[0.         0.         0.13534749 ... 0.04511583 0.         0.04511583]
 [0.         0.         0.         ... 0.         0.14757503 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.17333612 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [53]:
from gensim.models import Word2Vec
#for word2vec , data is required as list of lists
#more the data, better is the performance

ds = []
for e in corpus:
    ds.append(e.split(' '))

model = Word2Vec(ds,vector_size=150,window=25,min_count=2)    
# size is number of dimensions of vector that you want to build
# window is the maximum distance between two similar words
# min_count is the minimum number of times the word must appear in corpus
# workers is the numbers of cpu that you want to use

In [56]:
w1 = 'tasti'
print(model.wv.most_similar(w1))

[('delici', 0.6372284293174744), ('yummi', 0.5504572987556458), ('tastey', 0.5059869885444641), ('good', 0.4927912652492523), ('nice', 0.46648356318473816), ('snack', 0.4612698256969452), ('scrumptious', 0.4228350520133972), ('cruch', 0.4072302579879761), ('crunchi', 0.40108391642570496), ('fill', 0.39876511693000793)]
